In [119]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import librosa
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [2]:
train = pd.read_csv('C:/Users/денис/Desktop/AIChallenge/new2/train_gt.csv', header=None, names=['names', 'labels'])
test = pd.read_csv('C:/Users/денис/Desktop/AIChallenge/new2/test.csv', header=None, names=['names', 'labels'])

In [110]:
class Transformer():

    def __init__(self) -> None:
        self.sample_rate = 16000
        self.sc_1 = StandardScaler()
        self.sc_2 = StandardScaler()
        self.sc_3 = StandardScaler()
        self.sc_4 = StandardScaler()
        self.sc_5 = StandardScaler()
        self.sc_6 = StandardScaler()
        self.sc_7 = StandardScaler()
        self.princ_1 = PCA(n_components=1)
        self.princ_2 = PCA(n_components=5)
        self.princ_3 = PCA(n_components=2)
        self.princ_4 = PCA(n_components=2)
        self.princ_5 = PCA(n_components=2)
        self.princ_6 = PCA(n_components=2)
        self.princ_7 = PCA(n_components=2)
        self.path = "C:/Users/денис/Desktop/AIChallenge/new2"
        self.FRAME_SIZE = 256 
        self.HOP_LENGTH = 64 

    def mfccs_ob(self, audio):
        mfccs = librosa.feature.mfcc(y=audio, n_mfcc=13, sr=self.sample_rate)
        delta_mfccs = librosa.feature.delta(mfccs)
        delta2_mfcc = librosa.feature.delta(mfccs, order=2)
        mfccs_audio = np.concatenate((mfccs, delta_mfccs, delta2_mfcc))
        return mfccs_audio
    
    def mfccs_1(self, mfccs):
        self.sc_1.fit(mfccs)
        scale_1 = self.sc_1.transform(mfccs)
        x_1 = self.princ_1.fit_transform(scale_1)
        return [r[0] for r in x_1]
    
    def mfccs_2(self, frame_x):
        self.sc_2.fit(frame_x)
        scale_2 = self.sc_2.transform(frame_x)
        x_2 = self.princ_2.fit_transform(scale_2)
        return x_2
    
    def amplitude_envelope(self, audio, frame_size, hop_length):
        return np.array([max(audio[i:i+frame_size]) for i in range(0, audio.size, hop_length)])
    

    def general_process(self, frame, path_dop):
        spisok = frame['names'].tolist()

        spisok_add = []
        spisok_amp = []
        spisok_rms = []
        spisok_zcr = []
        spisok_cent = []
        spisok_down = []

        number_1 = 1

        for i in spisok:
            audio, _ = librosa.load(f'{self.path}/{path_dop}/{i}', sr=self.sample_rate)

            spisok_add.append(self.mfccs_1(self.mfccs_ob(audio)))
            spisok_amp.append(list(self.amplitude_envelope(audio, self.FRAME_SIZE, self.HOP_LENGTH)))
            spisok_rms.append(librosa.feature.rms(y=audio, frame_length=self.FRAME_SIZE, hop_length=self.HOP_LENGTH)[0])
            spisok_zcr.append(librosa.feature.zero_crossing_rate(y=audio, frame_length=self.FRAME_SIZE, hop_length=self.HOP_LENGTH)[0])
            spisok_cent.append(librosa.feature.spectral_centroid(y=audio, sr=self.sample_rate)[0])
            spisok_down.append(librosa.feature.spectral_rolloff(y=audio, sr=self.sample_rate)[0])

            print(f'm1/{number_1}')
            number_1+=1

        spisok_final = self.mfccs_2(np.array(spisok_add))

        spisok_mfccs_1 = [mfccs_1[0] for mfccs_1 in spisok_final]
        spisok_mfccs_2 = [mfccs_2[1] for mfccs_2 in spisok_final]
        spisok_mfccs_3 = [mfccs_3[2] for mfccs_3 in spisok_final]
        spisok_mfccs_4 = [mfccs_4[3] for mfccs_4 in spisok_final]
        spisok_mfccs_5 = [mfccs_5[4] for mfccs_5 in spisok_final]
        print('MFCCS is ready!')
        spisok_amp_1 = [np.std(amp) for amp in spisok_amp]
        spisok_amp_2 = [np.mean(amp) for amp in spisok_amp]
        print('AMP is ready!')
        spisok_rms_1 = [np.std(rms) for rms in spisok_rms]
        spisok_rms_2 = [np.mean(rms) for rms in spisok_rms]
        print('RMS is ready!')
        spisok_zcr_1 = [np.std(zcr) for zcr in spisok_zcr]
        spisok_zcr_2 = [np.mean(zcr) for zcr in spisok_zcr]
        print('ZCR is ready!')
        spisok_cent_1 = [np.std(cent) for cent in spisok_cent]
        spisok_cent_2 = [np.mean(cent) for cent in spisok_cent]
        print('CENT is ready!')
        spisok_down_1 = [np.std(down) for down in spisok_down]
        spisok_down_2 = [np.mean(down) for down in spisok_down]
        print('DOWN is ready!')

        data = pd.DataFrame({
            'mfccs_1': spisok_mfccs_1,
            'mfccs_2': spisok_mfccs_2,
            'mfccs_3': spisok_mfccs_3,
            'mfccs_4': spisok_mfccs_4,
            'mfccs_5': spisok_mfccs_5,
            'amp_std': spisok_amp_1,
            'amp_mean': spisok_amp_2,
            'rms_std': spisok_rms_1,
            'rms_mean': spisok_rms_2,
            'zcr_std': spisok_zcr_1,
            'zcr_mean': spisok_zcr_2,
            'cent_std': spisok_cent_1,
            'cent_mean': spisok_cent_2,
            'down_std': spisok_down_1,
            'down_mean': spisok_down_2
        })

        return data


tr = Transformer()

In [111]:
data =tr.general_process(train, "train")

m1/1
m1/2
m1/3
m1/4
m1/5
m1/6
m1/7
m1/8
m1/9
m1/10
m1/11
m1/12
m1/13
m1/14
m1/15
m1/16
m1/17
m1/18
m1/19
m1/20
m1/21
m1/22
m1/23
m1/24
m1/25
m1/26
m1/27
m1/28
m1/29
m1/30
m1/31
m1/32
m1/33
m1/34
m1/35
m1/36
m1/37
m1/38
m1/39
m1/40
m1/41
m1/42
m1/43
m1/44
m1/45
m1/46
m1/47
m1/48
m1/49
m1/50
m1/51
m1/52
m1/53
m1/54
m1/55
m1/56
m1/57
m1/58
m1/59
m1/60
m1/61
m1/62
m1/63
m1/64
m1/65
m1/66
m1/67
m1/68
m1/69
m1/70
m1/71
m1/72
m1/73
m1/74
m1/75
m1/76
m1/77
m1/78
m1/79
m1/80
m1/81
m1/82
m1/83
m1/84
m1/85
m1/86
m1/87
m1/88
m1/89
m1/90
m1/91
m1/92
m1/93
m1/94
m1/95
m1/96
m1/97
m1/98
m1/99
m1/100
m1/101
m1/102
m1/103
m1/104
m1/105
m1/106
m1/107
m1/108
m1/109
m1/110
m1/111
m1/112
m1/113
m1/114
m1/115
m1/116
m1/117
m1/118
m1/119
m1/120
m1/121
m1/122
m1/123
m1/124
m1/125
m1/126
m1/127
m1/128
m1/129
m1/130
m1/131
m1/132
m1/133
m1/134
m1/135
m1/136
m1/137
m1/138
m1/139
m1/140
m1/141
m1/142
m1/143
m1/144
m1/145
m1/146
m1/147
m1/148
m1/149
m1/150
m1/151
m1/152
m1/153
m1/154
m1/155
m1/156
m1/157
m1/158
m1

In [114]:
X_train, X_test, y_train, y_test = train_test_split(data, train['labels'].tolist(), test_size=0.2, random_state=42)

In [116]:
sc_train = StandardScaler()
sc_test = StandardScaler()

In [134]:
X_train = np.array(sc_train.fit_transform(X_train))
X_test = np.array(sc_test.fit_transform(X_test))
y_train = np.array(y_train)
y_test = np.array(y_test)

In [138]:
modeltf = Sequential([
    Dense(250, input_shape=(15,), activation='relu'),
    Dense(1024, activation='relu'),
    Dense(1024, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [139]:
modeltf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [140]:
modeltf.fit(X_train, y_train, batch_size=20, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
353/353 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.6168 - loss: 0.6710 - val_accuracy: 0.6405 - val_loss: 0.6587
Epoch 2/30
353/353 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6297 - loss: 0.6557 - val_accuracy: 0.6451 - val_loss: 0.6499
Epoch 3/30
353/353 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6442 - loss: 0.6427 - val_accuracy: 0.6417 - val_loss: 0.6554
Epoch 4/30
353/353 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6361 - loss: 0.6482 - val_accuracy: 0.6422 - val_loss: 0.6542
Epoch 5/30
353/353 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6460 - loss: 0.6339 - val_accuracy: 0.6246 - val_loss: 0.6542
Epoch 6/30
353/353 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6460 - loss: 0.6352 - val_accuracy: 0.6275 - val_loss: 0.6554
Epoch 7/30
353/353 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6407 - loss: 0.6342 - val_accuracy: 0.6184 - val_loss: 0.6586
Epoch 8/30
353/353 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.6528 - loss: 0.6247 - val_accu

In [141]:
loss_and_metrics = modeltf.evaluate(X_test, y_test)
print(loss_and_metrics)
print('Loss = ',loss_and_metrics[0])
print('Accuracy = ',loss_and_metrics[1])

56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5597 - loss: 2.5119
[2.3211050033569336, 0.5672913193702698]
Loss =  2.3211050033569336
Accuracy =  0.5672913193702698
